<a href="https://colab.research.google.com/github/j-min/Adversarial_Video_Summary/blob/master/custom_video_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Make sure GPU is used
!nvidia-smi

Tue Mar 28 17:23:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!unzip HiREST-master.zip
# git clone https://github.com/j-min/HiREST

Archive:  HiREST-master.zip
52bba440e2a49d3d7ac67fa6fc99d9d48c460b99
   creating: HiREST-master/
  inflating: HiREST-master/.gitignore  
   creating: HiREST-master/EVA_clip/
  inflating: HiREST-master/EVA_clip/CLIP.png  
  inflating: HiREST-master/EVA_clip/README.md  
  inflating: HiREST-master/EVA_clip/__init__.py  
  inflating: HiREST-master/EVA_clip/benchmark.md  
 extracting: HiREST-master/EVA_clip/bpe_simple_vocab_16e6.txt.gz  
  inflating: HiREST-master/EVA_clip/clip.py  
  inflating: HiREST-master/EVA_clip/eva_clip.py  
  inflating: HiREST-master/EVA_clip/eva_model.py  
  inflating: HiREST-master/EVA_clip/model.py  
   creating: HiREST-master/EVA_clip/model_configs/
  inflating: HiREST-master/EVA_clip/model_configs/EVA_CLIP_g_14.json  
  inflating: HiREST-master/EVA_clip/simple_tokenizer.py  
  inflating: HiREST-master/EVA_clip/vit_model.py  
  inflating: HiREST-master/LICENSE   
  inflating: HiREST-master/README.md  
  inflating: HiREST-master/args.py   
   creating: HiREST-mas

In [ ]:
cd HiREST-master

/content/HiREST-master


# Installation

First install required libraries and download all pretrained weights

In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git (to revision eff383b27b783e280c089475852ba83f20f64998) to /tmp/pip-req-build-skxraei_
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-skxraei_
  Running command git rev-parse -q --verify 'sha^eff383b27b783e280c089475852ba83f20f64998'
  Running command git fetch -q https://github.com/openai/whisper.git eff383b27b783e280c089475852ba83f20f64998
  Running command git checkout -q eff383b27b783e280c089475852ba83f20f64998
  Resolved https://github.com/openai/whisper.git to commit eff383b27b783e280c089475852ba83f20f64998
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/bckim92/language-evaluation.git to /tmp/pip-req-build-m9yegoqp
  Running command git clone --filter=blob:none --quiet https://github.com/bckim92/language-evaluation.git /tmp/pip-req-build-m9yegoqp
  R

# Download Checkpoints

In [ ]:
!mkdir ./checkpoints/
!mkdir ./checkpoints/hirest_joint_model/
!mkdir ./pretrained_weights/

In [ ]:
# Pretrained CLIP4Caption
!gdown https://drive.google.com/uc?id=17p476sL5_KZoQ2h4e1TU7-qWH4VqzReT
!mv clip4caption_vit-b-32_model.bin ./pretrained_weights/clip4caption_vit-b-32_model.bin

# EVA-CLIP-G (frozen visual/textual encoder)
!wget https://huggingface.co/BAAI/EVA/resolve/main/eva_clip_psz14.pt
!mv eva_clip_psz14.pt ./pretrained_weights/eva_clip_psz14.pt

# Finetuned HiREST joint baseline model
!wget https://huggingface.co/j-min/HiREST-baseline/resolve/main/HiREST_BEST.pth
!mv HiREST_BEST.pth ./checkpoints/hirest_joint_model/BEST.pth

Downloading...
From: https://drive.google.com/uc?id=17p476sL5_KZoQ2h4e1TU7-qWH4VqzReT
To: /content/HiREST-master/clip4caption_vit-b-32_model.bin
100% 236M/236M [00:02<00:00, 97.2MB/s]
--2023-03-28 17:25:47--  https://huggingface.co/BAAI/EVA/resolve/main/eva_clip_psz14.pt
Resolving huggingface.co (huggingface.co)... 52.2.178.255, 35.173.225.216, 34.203.133.210, ...
Connecting to huggingface.co (huggingface.co)|52.2.178.255|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/58/f1/58f17426d51756a1aa3793fad0651594b6c14d7bf99c577181cb2e0614f61817/fe1d3a90bb88c3816a8c708b8a16c11c93cc323ec9e7bc7a0e8b979449fa9785?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27eva_clip_psz14.pt%3B+filename%3D%22eva_clip_psz14.pt%22%3B&Expires=1680283548&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzU4L2YxLzU4ZjE3NDI2ZDUxNzU2YTFhYTM3OTNmYWQwNjUxNTk0YjZjMTRkN2JmOTljNTc3MTgxY2IyZTA2MTR

# Create Data Dirs

In [ ]:
!mkdir custom_video_pipeline/
!mkdir custom_video_pipeline/data
!mkdir custom_video_pipeline/data/eva_clip_features/
!mkdir custom_video_pipeline/data/ASR_feats_all-MiniLM-L6-v2/
!mkdir custom_video_pipeline/data/ASR/
!mkdir custom_video_pipeline/data/raw_frames/
!mkdir custom_video_pipeline/data/audio/
!mkdir custom_video_pipeline/data/splits/
!mkdir custom_video_pipeline/data/videos/

# Download a sample video

In [ ]:
!wget https://huggingface.co/j-min/HiREST-baseline/resolve/main/4Gx9W0XFAkA.mp4
!mv 4Gx9W0XFAkA.mp4 ./custom_video_pipeline/data/videos/4Gx9W0XFAkA.mp4

--2023-03-28 17:30:11--  https://huggingface.co/j-min/HiREST-baseline/resolve/main/4Gx9W0XFAkA.mp4
Resolving huggingface.co (huggingface.co)... 35.173.225.216, 34.203.133.210, 52.2.178.255, ...
Connecting to huggingface.co (huggingface.co)|35.173.225.216|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/43/e5/43e5477f8142d4d657272c2d83709064f52cea70061d7f463fe4e9973cc4707b/89fe3b10d4318445d42460c0614dc258a94707214e5498bc344eafc7d85131b8?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%274Gx9W0XFAkA.mp4%3B+filename%3D%224Gx9W0XFAkA.mp4%22%3B&response-content-type=video%2Fmp4&Expires=1680283813&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzQzL2U1LzQzZTU0NzdmODE0MmQ0ZDY1NzI3MmMyZDgzNzA5MDY0ZjUyY2VhNzAwNjFkN2Y0NjNmZTRlOTk3M2NjNDcwN2IvODlmZTNiMTBkNDMxODQ0NWQ0MjQ2MGMwNjE0ZGMyNThhOTQ3MDcyMTRlNTQ5OGJjMzQ0ZWFmYzdkODUxMzFiOD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSomcmVz

# Create Video Data Structure

In [ ]:
import json

In [ ]:
data = {
    "Make Butter Biscuits": {
        "4Gx9W0XFAkA.mp4": {
            "relevant": True,
            "clip": True,
            "v_duration": 299.73,
            "bounds": [ 0, 1 ],
            "steps": [ ]
        }
    }
}

with open("./custom_video_pipeline/data/splits/all_data_test.json", 'w') as f:
    json.dump(data, f)

# Extract ASR Features
*Note: Due to slight randomness with Whisper, ASR results may vary between runs

In [ ]:
!python ./extraction/whisper_ASR/extract_audio.py --workers 1 --data_path ./custom_video_pipeline/data/splits/all_data_test.json --save_path ./custom_video_pipeline/data/audio/ --video_path ./custom_video_pipeline/data/videos/

Extracting audio from videos...
Found 1 videos at custom_video_pipeline/data/videos
Will save audio to custom_video_pipeline/data/audio
Multiprocessing with 1 workers
  0% 0/1 [00:00<?, ?it/s]size=    9367kB time=00:04:59.74 bitrate= 256.0kbits/s speed= 659x    
100% 1/1 [00:01<00:00,  1.27s/it]


In [ ]:
!python ./extraction/whisper_ASR/extract_ASR.py --model small.en --audio_dir ./custom_video_pipeline/data/audio/ --asr_dir ./custom_video_pipeline/data/ASR/

Namespace(model='small.en', audio_dir='./custom_video_pipeline/data/audio/', asr_dir='./custom_video_pipeline/data/ASR/')
100%|███████████████████████████████████████| 461M/461M [00:13<00:00, 36.1MiB/s]
Model loaded:  small.en at device cuda:0
  0% 0/1 [00:00<?, ?it/s]2023-03-28 17:31:21.234631: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-28 17:31:21.234751: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-28 17:31:21.234773: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvi

In [ ]:
# ASR result
!head -n 20 ./custom_video_pipeline/data/ASR/4Gx9W0XFAkA.srt

1
00:00:00,000 --> 00:00:12,320
What's up YouTube, y'all know I'm good?

2
00:00:12,320 --> 00:00:13,320
You good?

3
00:00:13,320 --> 00:00:14,320
You good?

4
00:00:14,320 --> 00:00:15,320
I'm good, you good?

5
00:00:15,320 --> 00:00:16,320
Okay good.



In [ ]:
!python ./extraction/whisper_ASR/extract_ASR_embedding.py --model sentence-transformers/all-MiniLM-L6-v2 --asr_dir ./custom_video_pipeline/data/ASR/ --save_dir ./custom_video_pipeline/data/ASR_feats_all-MiniLM-L6-v2/

Namespace(model='sentence-transformers/all-MiniLM-L6-v2', asr_dir='./custom_video_pipeline/data/ASR/', save_dir='./custom_video_pipeline/data/ASR_feats_all-MiniLM-L6-v2/')
model loaded:  sentence-transformers/all-MiniLM-L6-v2
ASR_feats_dir:  custom_video_pipeline/data/ASR_feats_all-MiniLM-L6-v2
100% 1/1 [00:00<00:00,  1.50it/s]


# Extract Video Features

In [ ]:
# extract video frames in 1fps
!python ./extraction/video_features/extract_frames.py --workers 1 --data_path ./custom_video_pipeline/data/splits/all_data_test.json --save_path ./custom_video_pipeline/data/raw_frames/ --video_path ./custom_video_pipeline/data/videos/

Namespace(workers=1, data_path='./custom_video_pipeline/data/splits/all_data_test.json', save_path='./custom_video_pipeline/data/raw_frames/', video_path='./custom_video_pipeline/data/videos/')
CV2 version: 4.7.0
Loaded data from ./custom_video_pipeline/data/splits/all_data_test.json
Saving frames to custom_video_pipeline/data/raw_frames
100% 1/1 [00:00<00:00, 15420.24it/s]
Extracting frames from 1 videos
Using 1 worker
100% 1/1 [01:23<00:00, 83.81s/it]
Done


In [ ]:
# extract EVA-CLIP-G features
!python ./extraction/video_features/extract_features.py --frame_path ./custom_video_pipeline/data/raw_frames/ --save_path ./custom_video_pipeline/data/eva_clip_features/ --batch_size 10

Namespace(device=0, slice_start=0, slice_end=-1, frame_path='./custom_video_pipeline/data/raw_frames/', save_path='./custom_video_pipeline/data/eva_clip_features/', batch_size=10)
<All keys matched successfully>
100% 1/1 [01:00<00:00, 60.51s/it]


In [ ]:
!python ./extraction/video_features/check_feature_size.py --data_file ./custom_video_pipeline/data/splits/all_data_test.json --feature_folder ./custom_video_pipeline/data/eva_clip_features/

100% 1/1 [00:00<00:00, 614.82it/s]


# Run the Model End-to-End for all tasks:

moment_retrieval -> moment_segmentation -> step_captioning

In [ ]:
!python run.py \
        --data_dir './custom_video_pipeline/data/splits/' \
        --video_feature_dir './custom_video_pipeline/data/eva_clip_features' \
        --asr_dir './custom_video_pipeline/data/ASR' \
        --asr_feature_dir './custom_video_pipeline/data/ASR_feats_all-MiniLM-L6-v2' \
        --eval_batch_size 5 \
        --task_moment_retrieval \
        --task_moment_segmentation \
        --task_step_captioning \
        --ckpt_dir './checkpoints/hirest_joint_model/' \
        --end_to_end

Namespace(data_dir='./custom_video_pipeline/data/splits/', video_feature_dir='./custom_video_pipeline/data/eva_clip_features', asr_dir='./custom_video_pipeline/data/ASR', asr_feature_dir='./custom_video_pipeline/data/ASR_feats_all-MiniLM-L6-v2', task_moment_retrieval=True, task_moment_segmentation=True, task_step_captioning=True, use_gaussian_blur=False, train_batch_size=32, eval_batch_size=5, clip_grad_norm=-1.0, gradient_accumulation_steps=1, epochs=10, lr=1e-05, weight_decay=0.0, warmup_steps=0.1, seed=2222, optim='adamw', fp16=False, train=False, end_to_end=True, n_model_frames=-1, num_workers=4, distributed=False, device='cuda', num_beams=5, load=None, ckpt_dir='./checkpoints/hirest_joint_model/', comment='', num_thread_reader=16, n_display=16, max_frames_step_captioning=20, max_words=48, visual_num_hidden_layers=2, decoder_num_hidden_layers=2, local_rank=0, gpu=0, world_size=1)
Process Launching at GPU 0
NLGEval not installed, if you want to train CLIP4Caption,
          please i

# Print Final Results

In [ ]:
with open("./checkpoints/hirest_joint_model/final_end_to_end_results.json", 'r') as f:
    results = json.load(f)

for prompt in results:
    print("Prompt:", prompt)
    for video in results[prompt]:
        print("Video:", video)

        print("Retrieved Moment:", results[prompt][video]["bounds"])
        
        for step in results[prompt][video]["steps"]:
            print(f"Segment {step['index']}:", step["absolute_bounds"], step["heading"])

Prompt: Make Butter Biscuits
Video: 4Gx9W0XFAkA.mp4
Retrieved Moment: [35, 250]
Segment 0: [35, 47] mix it
Segment 1: [47, 52] mix it
Segment 2: [52, 64] mix it
Segment 3: [64, 73] bake it in oven
Segment 4: [73, 78] transfer into plate
Segment 5: [78, 93] mix it well
Segment 6: [93, 99] mix it
Segment 7: [99, 113] mix it
Segment 8: [113, 122] mix it
Segment 9: [122, 129] bake it in oven
Segment 10: [129, 138] repeat the process
Segment 11: [138, 143] repeat the process
Segment 12: [143, 161] roll it out
Segment 13: [161, 166] put it on plate
Segment 14: [166, 172] mix it well
Segment 15: [172, 180] bake it in oven
Segment 16: [180, 187] cut it out


# Compare with original video

In [ ]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/4Gx9W0XFAkA" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>